## 1단계. 필수 라이브러리 설치

In [ ]:
!pip install transformers langchain chromadb fastapi uvicorn huggingface_hub mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:00

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


## 2단계. 필수 라이브러리 임포트 및 로깅 설정

In [ ]:
import os
import logging
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import mysql.connector

# 로깅 설정
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

## 3단계. Hugging Face 로그인 및 모델 설정

In [ ]:
# Hugging Face 로그인
from huggingface_hub import login
login(token="")  # your_huggingface_api_token을 복사한 토큰으로 대체하세요.

# LLM 모델 및 파이프라인 설정
logging.info("Gemma-2b 모델 및 임베딩 모델 로드 중...")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", torch_dtype="auto").cuda()

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_new_tokens=150
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 4단계. MYSQL 데이터 로드

GCP 상 MySQL에 접근하기 위해서는 현재 작업중인 Colab의 IP 주소를 파악하고, GCP 방화벽 규칙에 추가해야 한다.

In [ ]:
!curl ipecho.net/plain

34.83.68.106

In [ ]:
# MySQL에서 논문 데이터 가져오기
logging.info("MySQL 데이터베이스 연결 및 데이터 로드...")

try:
    db_connection = mysql.connector.connect(
        host="34.41.50.12",  # GCP MySQL 호스트
        user="root",        # MySQL 사용자명
        password="1234",    # MySQL 비밀번호
        database="mywebsite"   # MySQL 데이터베이스명
    )
    cursor = db_connection.cursor()

    try:
        cursor.execute("SELECT `index`, `title`, `abstract` FROM ARTICLE;")
        papers = cursor.fetchall()

        # 쿼리 실행 결과 로깅
        logging.info(f"가져온 데이터 개수: {len(papers)}")
        logging.info(f"가져온 데이터 일부: {papers[:10]}")  # 처음 10개 데이터만 출력

    except mysql.connector.Error as e:
        # 예외 처리 로깅
        logging.error(f"MySQL 쿼리 실행 실패: {e}")
        logging.error(f"실행 쿼리: SELECT `index`, `title`, `abstract` FROM ARTICLE;")
        raise RuntimeError("MySQL 쿼리 실행 실패")

except mysql.connector.Error as e:
    logging.error(f"MySQL 연결 실패: {e}")
    raise RuntimeError("MySQL 연결 실패")

finally:
    try:
        if cursor:
            cursor.close()
        if db_connection:
            db_connection.close()
    except Exception as e:
        logging.error(f"MySQL 연결 종료 실패: {e}")

ERROR:root:MySQL 연결 실패: 2003: Can't connect to MySQL server on '34.41.50.12:3306' (Errno 110: Connection timed out)
ERROR:root:MySQL 연결 종료 실패: name 'cursor' is not defined


RuntimeError: MySQL 연결 실패

## 5단계. 데이터 임베딩 및 ChromaDB 생성

In [ ]:
# 데이터 임베딩 및 ChromaDB 생성
logging.info("데이터 임베딩 및 ChromaDB 초기화...")
documents = [f"{title}. {abstract}" for _, title, abstract in papers]
ids = [str(id) for id, _, _ in papers]

try:
    vectorstore = Chroma.from_texts(
        texts=documents,
        embedding=embedding_model,
        ids=ids,
        persist_directory="db"
    )
    vectorstore.persist()
    logging.info("ChromaDB 저장 완료.")
except Exception as e:
    logging.error(f"ChromaDB 생성 실패: {e}")
    raise RuntimeError("ChromaDB 생성 실패")

## 6단계. 질의 및 응답 처리 루프

In [ ]:
# 질의 및 응답 처리
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 사용자 입력 및 처리 루프
while True:
    try:
        query = input("질문을 입력하세요 ('종료'를 입력하면 종료됩니다): ").strip()
        if query.lower() == "종료":
            print("프로그램을 종료합니다.")
            break

        # 검색 및 결과 출력
        logging.info(f"사용자 입력: {query}")
        search_results = retriever.vectorstore.similarity_search_with_score(query, k=3)

        # 검색된 문서와 점수 출력
        print("\n[검색된 문서 및 유사도 점수]")
        for i, (doc, score) in enumerate(search_results, start=1):
            print(f"{i}. 문서: {doc.page_content}")
            print(f"   스코어: {score:.4f}")

        # 결합된 문맥으로 LLM 호출
        combined_context = "\n".join([doc.page_content for doc, _ in search_results])
        input_data = {"query": query, "context": combined_context}
        response = qa_chain(input_data)

        # 답변 출력
        print(f"\n[응답]\n{response['result']}\n")
    except Exception as e:
        logging.error(f"오류 발생: {e}")
        print(f"오류 발생: {e}")

질문을 입력하세요 ('종료'를 입력하면 종료됩니다): Trade Wars

[검색된 문서 및 유사도 점수]
1. 문서: Trade Wars with Trade Deficits. 
   스코어: 0.3642
2. 문서: Trade, Trees, and Lives. 
   스코어: 0.9463
3. 문서: A dynamic auction for multilateral collaboration. 
   스코어: 1.1693


<ipython-input-12-8fad36c399ca>:26: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain(input_data)



[응답]
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Trade Wars with Trade Deficits. 

Trade, Trees, and Lives. 

A dynamic auction for multilateral collaboration. 

Question: Trade Wars
Helpful Answer: 
* Trade wars are a form of protectionism.
* Trade wars can lead to higher prices for consumers.
* Trade wars can harm the global economy.
* Trade wars can lead to retaliatory tariffs.
* Trade wars can lead to a decrease in trade.
* Trade wars can lead to a decrease in investment.
* Trade wars can lead to a decrease in economic growth.


**Question:** What is the impact of trade wars on the global economy?

**Answer:** Trade wars can harm the global economy. 


질문을 입력하세요 ('종료'를 입력하면 종료됩니다): look up for AI model

[검색된 문서 및 유사도 점수]
1. 문서: Artificial Intelligence in Financial Forecasting: Analyzing the Suitability of AI Models for Dollar/TL Exchange Rate Predictions. 
   스코어